##**Juntar la información de trabajo en un solo Dataframe**

# **Información conectada a datos abiertos**

La Ley 1273 de 2009 en Colombia introdujo el Título VII-A al Código Penal, tipificando delitos informáticos como el acceso abusivo a sistemas informáticos (art. 269A), la obstaculización de redes (art. 269B), la interceptación de datos (art. 269C) y la violación de datos personales (art. 269F). Esta ley respondió a la creciente dependencia de la tecnología digital, estableciendo un marco jurídico para proteger la información. Además, Colombia ratificó el Convenio de Budapest sobre Ciberdelincuencia a través de la Ley 1331 de 2009, lo que reforzó la cooperación internacional en la lucha contra la ciberdelincuencia.

Sin embargo, el rápido aumento de la conectividad a internet, impulsado por planes como Vive Digital y la Transformación Digital del Estado, ha generado nuevos desafíos. Aunque estas iniciativas buscan fomentar el desarrollo digital, no han ido acompañadas de mejoras proporcionales en la prevención, control y judicialización de delitos informáticos. Las regiones con brechas institucionales y limitaciones en ciberseguridad son especialmente vulnerables a estos delitos.

La falta de estudios que analicen la relación entre la cobertura de internet y la incidencia de delitos informáticos crea un vacío importante en el conocimiento sobre los riesgos digitales en Colombia. Esta carencia dificulta la dimensionar adecuadamente los riesgos y la capacidad estatal de respuesta ante la ciberdelincuencia, lo que podría estar favoreciendo el subregistro de delitos.

Por ello, es necesario realizar un análisis descriptivo y exploratorio para determinar si existe una correlación entre el aumento de la conectividad y el comportamiento de los delitos informáticos. Este estudio permitirá identificar tipologías de delitos, brechas en el reporte y mejorar las políticas de prevención y control

# **Cargar Información del geoportal**

In [ ]:
# ===================================
# 1. LIBRERÍAS
# ===================================
import pandas as pd
import chardet

# ===================================
# 2. CARGAR DATOS DE FORMA MANUAL
# ===================================
#from google.colab import files
#uploaded = files.upload()
df_geoespacio = pd.read_excel('Geoportal del DANE - Codificación Divipola.xlsx')

# ===================================
# 3. AJUSTAR DATOS
# ===================================
# Renombrando títulos
df_geoespacio.columns = df_geoespacio.iloc[0].values

# Eliminar la fila 1
df = df_geoespacio.drop(0).reset_index(drop=True)

#Agregar columna llave
df_geoespacio['Dep_Mpio'] = df_geoespacio ['Nombre Departamento'] + df_geoespacio ['Nombre Municipio']

#Quitar duplicados
df_geoespacio.drop_duplicates(subset= 'Dep_Mpio',inplace=True)

#Colocar la llave como Indice
df_geoespacio.set_index ('Dep_Mpio', inplace= True)

#Eliminar columnas inncesarias
df_geoespacio.drop (['Nombre Centro Poblado', 'Tipo Centro Poblado', 'Nombre Distrito', 'Municipio/Áreas No Municipalizadas (ANM)', 'Nombre Área Metropolitana', 'Código Centro Poblado', 'Longitud', 'Latitud'], axis = 1, inplace= True)
df_geoespacio

Saving Geoportal del DANE - Codificación Divipola.xlsx to Geoportal del DANE - Codificación Divipola.xlsx


,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio
Dep_Mpio,,,,
Nombre DepartamentoNombre Municipio,Código Departamento,Código Municipio,Nombre Departamento,Nombre Municipio
ANTIOQUIAMEDELLÍN,05,05001,ANTIOQUIA,MEDELLÍN
ANTIOQUIAABEJORRAL,05,05002,ANTIOQUIA,ABEJORRAL
ANTIOQUIAABRIAQUÍ,05,05004,ANTIOQUIA,ABRIAQUÍ
ANTIOQUIAALEJANDRÍA,05,05021,ANTIOQUIA,ALEJANDRÍA
...,...,...,...,...
VAUPÉSYAVARATÉ (ANM),97,97889,VAUPÉS,YAVARATÉ (ANM)
VICHADAPUERTO CARREÑO,99,99001,VICHADA,PUERTO CARREÑO
VICHADALA PRIMAVERA,99,99524,VICHADA,LA PRIMAVERA


In [ ]:
#df_geoespacio.to_excel("resultados.xlsx", index=False)

# **Carga de la tabla de delitos**

In [ ]:
# ===================================
# 1. LIBRERÍAS
# ===================================
import pandas as pd
import requests

# ===================================
# 2. DESCARGAR DATOS DELITOS
# ===================================
base_url = "https://www.datos.gov.co/resource/6d52-qyqg.json"
anios = [2022, 2023, 2024]
lote = 1000
datos = []

print("Iniciando descarga filtrada por año y país...")

for anio in anios:
    offset = 0
    while True:
        params = {
            "$limit": lote,
            "$offset": offset,
            "$where": f"a_o_hechos = '{anio}' AND pais_hecho = 'Colombia' AND grupo_delito = 'DELITOS INFORMATICOS'"
        }
        response = requests.get(base_url, params=params)
        if response.status_code != 200:
            print(f"Error {response.status_code} en año {anio}, offset {offset}")
            break

        data = response.json()
        if not data:
            break

        datos.extend(data)
        offset += lote
        print(f"Año {anio} - registros descargados: {offset}")

print("Descarga completa.")
df_delitos = pd.DataFrame(datos)
print(f"\nTotal de registros filtrados: {len(df_delitos)}")

# ===================================
# 3. RENOMBRAR COLUMNAS
# ===================================
nuevos_nombres_columna = {
    "criminalidad" :  "Criminalidad",
    "es_archivo" :  "Es_Archivo",
    "es_preclusion" :  "Es_Preclusion",
    "estado" :  "Estado",
    "etapa_caso" :  "Etapa_Caso",
    "ley" :  "Ley",
    "departamento_hecho" :  "Departamento_Hecho",
    "municipio_hecho" :  "Municipio_Hecho",
    "seccional" :  "Seccional",
    "a_o_hechos" :  "Año_Hechos",
    "a_o_entrada" :  "Año_Entrada",
    "a_o_denuncia" :  "Año_Denuncia",
    "delito" :  "Delito",
    "grupo_delito" :  "Grupo_Delito",
    "consumado" :  "Consumado"
}
df_delitos.rename(columns=nuevos_nombres_columna, inplace=True)

# ===================================
# 4. CARGAR ARCHIVO DE CORRECCIONES
# ===================================
#from google.colab import files
#uploaded = files.upload()
df_correcciones = pd.read_excel("Nuevos_Dep_Mpio.xlsx")

# Normalizar texto para unir correctamente
for col in ['Departamento_Hecho', 'Municipio_Hecho']:
    df_delitos[col] = df_delitos[col].astype(str).str.upper().str.strip()

for col in ['Nombre Departamento', 'Nombre Municipio']:
    df_correcciones[col] = df_correcciones[col].astype(str).str.upper().str.strip()

# ===================================
# 5. UNIR Y REEMPLAZAR DATOS
# ===================================
df_delitos = df_delitos.merge(
    df_correcciones,
    left_on=['Departamento_Hecho', 'Municipio_Hecho'],
    right_on=['Nombre Departamento', 'Nombre Municipio'],
    how='left'
)

# Usar las columnas corregidas si existen, o mantener las originales
df_delitos['Departamento_Final'] = df_delitos['Departamento_Corregido'].combine_first(df_delitos['Departamento_Hecho'])
df_delitos['Municipio_Final'] = df_delitos['Municipio_Corregido'].combine_first(df_delitos['Municipio_Hecho'])

# Crear nueva llave corregida
df_delitos['Dep_Mpio'] = df_delitos['Departamento_Final'] + df_delitos['Municipio_Final']

#Borrar columnas innecesarias
df_delitos = df_delitos.drop (['Departamento_Hecho', 'Municipio_Hecho', 'pais_hecho', 'Departamento_Final', 'Municipio_Final', 'Nombre Departamento', 'Nombre Municipio'], axis = 1)

#Renombrar Departamento y Municipio
nuevos_nombres_columna = {
    "Departamento_Corregido" :  "Departamento_Hecho",
    "Municipio_Corregido" :  "Municipio_Hecho"
}
df_delitos.rename(columns=nuevos_nombres_columna, inplace=True)

#Colocar la llave como Indice
df_delitos.set_index ('Dep_Mpio', inplace= True)

# ===================================
# 6. UNIR CON TABLA GEOESPACIO
# ===================================
df_delitos = df_delitos.join (df_geoespacio)

#Borrar columnas innecesarias
df_delitos = df_delitos.drop (['Nombre Departamento', 'Nombre Municipio'], axis = 1)

#Convertir columnas numéricas a texto
df_delitos['Código Departamento'] = df_delitos['Código Departamento'].astype(int).astype(str)
df_delitos['Código Municipio'] = df_delitos['Código Municipio'].astype(int).astype(str)


#Crear nueva columna de codigo departamento y municipio
df_delitos['Cod__Anno_Dep_Mpio'] = df_delitos['Año_Hechos'].astype(str) + df_delitos['Código Departamento'] + df_delitos['Código Municipio']

# ===================================
# 7. CAMBIAR INDICE
# ===================================
df_delitos.set_index('Cod__Anno_Dep_Mpio', inplace= True)

Iniciando descarga filtrada por año y país...
Año 2022 - registros descargados: 1000
Año 2022 - registros descargados: 2000
Año 2022 - registros descargados: 3000
Año 2022 - registros descargados: 4000
Año 2022 - registros descargados: 5000
Año 2022 - registros descargados: 6000
Año 2022 - registros descargados: 7000
Año 2022 - registros descargados: 8000
Año 2023 - registros descargados: 1000
Año 2023 - registros descargados: 2000
Año 2023 - registros descargados: 3000
Año 2023 - registros descargados: 4000
Año 2023 - registros descargados: 5000
Año 2023 - registros descargados: 6000
Año 2023 - registros descargados: 7000
Año 2023 - registros descargados: 8000
Año 2024 - registros descargados: 1000
Año 2024 - registros descargados: 2000
Año 2024 - registros descargados: 3000
Año 2024 - registros descargados: 4000
Año 2024 - registros descargados: 5000
Año 2024 - registros descargados: 6000
Año 2024 - registros descargados: 7000
Año 2024 - registros descargados: 8000
Descarga completa.

Saving Nuevos_Dep_Mpio.xlsx to Nuevos_Dep_Mpio.xlsx


In [ ]:
#df_delitos.to_excel("delitos.xlsx", index= True)

,Criminalidad,Es_Archivo,Es_Preclusion,Estado,Etapa_Caso,Ley,Seccional,Año_Hechos,Año_Entrada,Año_Denuncia,Delito,Grupo_Delito,Consumado,total_procesos,Departamento_Hecho,Municipio_Hecho,Código Departamento,Código Municipio
Cod_Anno_Dep_Mpio,,,,,,,,,,,,,,,,,,
202255360,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE MEDELLÍN,2022,2022,2022,HURTO POR MEDIOS INFORMATICOS Y SEMEJANTES ART...,DELITOS INFORMATICOS,NO APLICA,161,ANTIOQUIA,ITAGÜÍ,5,5360
20222323417,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE CÓRDOBA,2022,2022,2022,ACCESO ABUSIVO A UN SISTEMA INFORMATICO ART 26...,DELITOS INFORMATICOS,NO APLICA,5,CÓRDOBA,LORICA,23,23417
20221111001,SI,NO,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE BOGOTÁ,2022,2022,2022,TRANSFERENCIA NO CONSENTIDA DE ACTIVOS VALIEND...,DELITOS INFORMATICOS,NO APLICA,17,NaN,NaN,11,11001
20227676001,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE CALI,2022,2022,2022,"INTERCEPTACION DE DATOS INFORMATICOS, ART 269C...",DELITOS INFORMATICOS,NO APLICA,6,VALLE DEL CAUCA,SANTIAGO DE CALI,76,76001
20227676622,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE VALLE DEL CAUCA,2022,2022,2022,VIOLACION DE DATOS PERSONALES ART 269F LEY 127...,DELITOS INFORMATICOS,NO APLICA,5,VALLE DEL CAUCA,ROLDANILLO,76,76622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20241818785,NO,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE CAQUETÁ,2024,2024,2024,HURTO POR MEDIOS INFORMATICOS Y SEMEJANTES ART...,DELITOS INFORMATICOS,NO APLICA,1,CAQUETÁ,SOLITA,18,18785
20245050370,SI,SI,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE META,2024,2024,2024,ACCESO ABUSIVO A UN SISTEMA INFORMATICO ART 26...,DELITOS INFORMATICOS,NO APLICA,1,META,URIBE,50,50370
20241919824,SI,NO,NO,INACTIVO,INDAGACIÓN,Ley 906,DIRECCIÓN SECCIONAL DE CAUCA,2024,2024,2024,VIOLACION DE DATOS PERSONALES ART 269F LEY 127...,DELITOS INFORMATICOS,NO APLICA,1,CAUCA,TOTORÓ,19,19824


# **Cargar tabla de Conectividad y Censo Dane**

In [ ]:
# ===================================
# 1. LIBRERÍAS
# ===================================
import pandas as pd
import chardet

# ===================================
# 2. CARGAR DATOS DE FORMA MANUAL
# ===================================
#from google.colab import files
#uploaded = files.upload()
df_conectividad_censo = pd.read_excel('TIC en Colombia Vs Censo.xlsx', sheet_name= '3')

# ===================================
# 3. RENOMBRAR TITULO Y BORRAR FILAS INNECESARIAS
# ===================================
# Renombrando títulos
df_conectividad_censo.columns = df_conectividad_censo.iloc[4].values

# Eliminar los primeros 5 registros
df_conectividad_censo = df_conectividad_censo.drop([0, 1, 2, 3, 4]).reset_index(drop=True)

# ===================================
# 4. RENOMBRAR COLUMNAS
# ===================================
# Cambiar por posición
nuevos_nombres = list(df_conectividad_censo.columns)

nuevos_nombres[0] = 'Año'
nuevos_nombres[1] = 'Trimestre'
nuevos_nombres[2] = 'Código Departamento' # Primer 'CÓDIGO DANE'
nuevos_nombres[3] = 'Departamento'
nuevos_nombres[4] = 'Código Municipio'    # Segundo 'CÓDIGO DANE'
nuevos_nombres[5] = 'Municipio'
nuevos_nombres[6] = 'No. Accesos Fijos A Internet'
nuevos_nombres[7] = 'Población Dane'
nuevos_nombres[8] = 'Penetración (%)'

df_conectividad_censo.columns = nuevos_nombres

# ===================================
# 5. AJUSTAR DATOS
# ===================================
#Cambiar de posisción columnas
df_conectividad_censo[['Población Dane', 'No. Accesos Fijos A Internet']] = \
df_conectividad_censo[['No. Accesos Fijos A Internet', 'Población Dane']]

#Agrupar por promedio de Poblacion Dane y No. Accesos Fijos A Internet
df_conectividad_censo = df_conectividad_censo.groupby(['Año', 'Código Departamento', 'Departamento', 'Código Municipio', 'Municipio'], as_index=False).agg({'Población Dane': 'mean', 'No. Accesos Fijos A Internet': 'mean'})

#Convertir columnas numéricas a texto
df_conectividad_censo['Código Departamento'] = df_conectividad_censo['Código Departamento'].astype(int).astype(str)
df_conectividad_censo['Código Municipio'] = df_conectividad_censo['Código Municipio'].astype(int).astype(str)

#Agregar columna llave
df_conectividad_censo['Cod__Anno_Dep_Mpio'] = df_conectividad_censo ['Año'].astype(str) + df_conectividad_censo ['Código Departamento'] + df_conectividad_censo ['Código Municipio']

#Agregar índice Cod_Dep_Mpio
df_conectividad_censo.set_index('Cod__Anno_Dep_Mpio', inplace= True)

In [ ]:
#df_conectividad_censo.to_excel("conetividad_censo.xlsx", index= True)

In [ ]:
df_conectividad_censo

,Año,Código Departamento,Departamento,Código Municipio,Municipio,Población Dane,No. Accesos Fijos A Internet
Cod__Anno_Dep_Mpio,,,,,,,
202255001,2022,5,ANTIOQUIA,5001,MEDELLÍN,753015.0,2572350.0
202255002,2022,5,ANTIOQUIA,5002,ABEJORRAL,2374.666667,21109.0
202255004,2022,5,ANTIOQUIA,5004,ABRIAQUÍ,151.0,2804.0
202255021,2022,5,ANTIOQUIA,5021,ALEJANDRÍA,261.0,4874.0
202255030,2022,5,ANTIOQUIA,5030,AMAGÁ,3565.333333,31851.0
...,...,...,...,...,...,...,...
20249797889,2024,97,VAUPÉS,97889,YAVARATÉ,11.75,1233.0
20249999001,2024,99,VICHADA,99001,PUERTO CARREÑO,1934.5,22614.0
20249999524,2024,99,VICHADA,99524,LA PRIMAVERA,163.0,11205.0


# **Unión de tablas agrupadas Censo y Conectividad - Tabla Delitos**

In [ ]:
# ===================================
# 1. LIBRERÍAS
# ===================================
import pandas as pd
import chardet

# ===================================
# 2. TRAER TABLA DELITOS Y AGRUPAR
# ===================================
df_delitos_agrupada = df_delitos

#Cambiar a entero columna Total Procesos
df_delitos_agrupada['total_procesos'] = df_delitos_agrupada['total_procesos'].astype(int)
df_delitos_agrupada = df_delitos_agrupada.groupby (['Cod__Anno_Dep_Mpio'])['total_procesos'].sum()

#Convertir df_delitos_agrupada en dataframe
df_delitos_agrupada = df_delitos_agrupada.to_frame()

# =============================================
# 3. UNIR TABLAS CENSO CONECTIVIDAD - DELITOS
# =============================================
df_union_cc_td = df_delitos_agrupada.join(df_conectividad_censo)

#Redondear columnas
df_union_cc_td['Población Dane'] = df_union_cc_td['Población Dane'].astype(float).round().astype(int)
df_union_cc_td['No. Accesos Fijos A Internet'] = df_union_cc_td['No. Accesos Fijos A Internet'].astype(float).round().astype(int)

In [ ]:
#df_union_cc_td.to_excel('union.xlsx', index=False)
df_union_cc_td

,total_procesos,Año,Código Departamento,Departamento,Código Municipio,Municipio,Población Dane,No. Accesos Fijos A Internet
Cod__Anno_Dep_Mpio,,,,,,,,
20221111001,20842,2022,11,BOGOTÁ D.C.,11001,"BOGOTÁ, D.C.",2260104,7873316
20221313001,1345,2022,13,BOLÍVAR,13001,CARTAGENA,193274,1043185
20221313006,1,2022,13,BOLÍVAR,13006,ACHÍ,133,26741
20221313042,3,2022,13,BOLÍVAR,13042,ARENAL,564,7853
20221313052,21,2022,13,BOLÍVAR,13052,ARJONA,2797,74773
...,...,...,...,...,...,...,...,...
20249797001,7,2024,97,VAUPÉS,97001,MITÚ,646,34886
20249797161,1,2024,97,VAUPÉS,97161,CARURU,36,3600
20249999001,18,2024,99,VICHADA,99001,PUERTO CARREÑO,1934,22614
